# SWIG Python-C++ google/s2geometry bindings

In [ ]:
%matplotlib inline

In [ ]:
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from ipywidgets import Checkbox, IntSlider, interactive
import matplotlib.pyplot as plt
from s2 import S2Cell, S2LatLng, S2LatLngRect, S2RegionCoverer
from shapely.geometry import Polygon

In [ ]:
def plot(min_level, max_level, max_cells, show_bbox=True):

    plt.figure(figsize=(4, 3), dpi=200)

    proj = cimgt.GoogleTiles()
    ax = plt.axes(projection=proj.crs)
    ax.add_image(proj, 17)

    # cartopy extent order is (x0, x1, y0, y1) or (lon0, lon1, lat0, lat1)
    ax.set_extent([-3.478665, -3.471724, 50.726045, 50.728911])

    # our bounding box of interest
    lat0, lon0 = 50.726771, -3.476471
    lat1, lon1 = 50.728089, -3.473832

    # shapely order is (x, y) or (lon, lat)
    bbox = Polygon([(lon0, lat0), (lon0, lat1),
                    (lon1, lat1), (lon1, lat0)])

    # s2 order is (y, x) or (lat, lon)
    region_rect = S2LatLngRect(
        S2LatLng.FromDegrees(lat0, lon0),
        S2LatLng.FromDegrees(lat1, lon1))

    coverer = S2RegionCoverer()
    coverer.set_min_level(min_level)
    coverer.set_max_level(max_level)
    coverer.set_max_cells(max_cells)
    covering = coverer.GetCovering(region_rect)

    geoms = []
    for cellid in covering:
        new_cell = S2Cell(cellid)
        vertices = []
        for i in range(0, 4):
            vertex = new_cell.GetVertex(i)
            latlng = S2LatLng(vertex)
            # shapely vertex order is (x, y) or (lon, lat)
            vertices.append((latlng.lng().degrees(),
                             latlng.lat().degrees()))
        geo = Polygon(vertices)
        geoms.append(geo)

    print("Total S2 geometries = {}".format(len(geoms)))

    pc = ccrs.PlateCarree()
    show_bbox = show_bbox.value if hasattr(show_bbox, "value") else show_bbox
    if show_bbox:
        ax.add_geometries([bbox], pc, facecolor=(0, 1, 0, .2), edgecolor=(0, 0, 0, 1))
    ax.add_geometries(geoms, pc, facecolor='coral', edgecolor='black', alpha=0.4)
    
    plt.show()

In [ ]:
def validate_min_level(*args):
    if min_level.value > max_level.value:
        min_level.value = args[0]['old']


def validate_max_level(*args):
    if max_level.value < min_level.value:
        max_level.value = args[0]['old']


min_level = IntSlider(min=0, max=30, step=1, value=4, continuous_update=False)
max_level = IntSlider(min=0, max=30, step=1, value=18, continuous_update=False)
max_cells = IntSlider(min=1, max=250, step=1, value=10, continuous_update=False)
bbox = Checkbox(value=True, description="Show bbox", disable=False, indent=False)

min_level.observe(validate_min_level, "value")
max_level.observe(validate_max_level, "value")

plotter = interactive(plot, min_level=min_level,
                      max_level=max_level,
                      max_cells=max_cells,
                      bbox=bbox)

output = plotter.children[-1]
output.layout.height = '500px'

In [ ]:
plotter